In [101]:
import aiohttp
import http
from bs4 import BeautifulSoup
import asyncio
import requests
import pandas as pd
import re, time
import pickle
import json
import steam_tags_f as stf

In [102]:
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)

In [103]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


GETTING THE TAGS AND MULTIPLAYER INFO

In [105]:
url = "http://api.steampowered.com/ISteamApps/GetAppList/v2/"

token = "1AC3575ADB761319F3EF72D1FA362232"

params = {"key":token}
response = requests.get(url, params=params)

data = response.json()

app_dict = {app['name']: app['appid'] for app in data['applist']['apps']}


In [106]:
top1000 = pd.read_csv('TOP 1000 Games cleaned.csv')

In [107]:
top1000["APP ID"] = 0

In [108]:
#top1000_2 = stf.add_appid(top1000,app_dict)

In [109]:
#top1000.to_csv('TOP 1000 Games cleaned AppId.csv', index=False)

In [110]:
#manually entered some app-ids that were not fetched
appid_fix = pd.read_csv('top_1000_games_cleaned_appid_fix.csv')

In [111]:
appid_fix

,Game Title,Steam App ID
0,PUBG: BATTLEGROUNDS,578080
1,Black Myth: Wukong,2358720
2,Palworld,1623730
3,Counter-Strike 2,730
4,Lost Ark,1599340
...,...,...
995,Winter Memories,2495450
996,Magicraft,2103140
997,Assetto Corsa Competizione,805550
998,Timberborn,1062090


In [112]:
txt = pd.read_csv(r"top_1000_games_cleaned_appid_fix.csv")
txt = txt.rename({"Game Title":"title", "Steam App ID":"id"}, axis=1)
appid_list = list(txt["id"])

In [113]:
# Example
appid = 2923300

# Step 1: Make a request to the Steam API
response = requests.get(f"http://store.steampowered.com/api/appdetails?appids={appid}")

# Step 2: Parse the response
game = response.json()

#Step 3: Pretty-print the JSON response for better readability
print(json.dumps(game, indent=4))  # Format with indentation of 4 spaces


{
    "2923300": {
        "success": true,
        "data": {
            "type": "game",
            "name": "Banana",
            "steam_appid": 2923300,
            "required_age": 0,
            "is_free": true,
            "detailed_description": "<p class=\"bb_paragraph\"><strong>About the Game</strong> </p><p class=\"bb_paragraph\">Banana is a clicker Game, in which you click a Banana!</p><p class=\"bb_paragraph\">Get a random banana every 3 hours and grow your collection! </p><p class=\"bb_paragraph\"></p><p class=\"bb_paragraph\">3 hour drop pool chances*: </p><ul class=\"bb_ul\"><li><p class=\"bb_paragraph\"> 54% Normal </p></li><li><p class=\"bb_paragraph\"> 30% Common</p></li><li><p class=\"bb_paragraph\">10% Uncommon</p></li><li><p class=\"bb_paragraph\">4% Rare</p></li><li><p class=\"bb_paragraph\">2% Epic</p></li><li><p class=\"bb_paragraph\"> 1/4 000 000 Ultra Rare </p></li><li><p class=\"bb_paragraph\"> 1/28 000 000 Legendary </p></li></ul><p class=\"bb_paragraph\">* S

In [114]:
genres = game["2923300"]["data"]["genres"]
descriptions = [genre["description"] for genre in genres]
print (descriptions)

['Adventure', 'Casual', 'Simulation', 'Strategy', 'Free To Play']


In [115]:
result = stf.check_multiplayer_single("1599340")
print(result) 

{'1599340': ['Multi-player', 'Co-op']}


In [116]:
#tag_dict = stf.get_tags(appid_list)

In [117]:
with open ("tag_dict.pkl", "rb") as tag_dict_file:
    tag_dict = pickle.load(tag_dict_file)


In [118]:
tag_dict

{578080: ['Action', 'Adventure', 'Massively Multiplayer', 'Free To Play'],
 2358720: ['Action', 'Adventure', 'RPG'],
 1623730: ['Action', 'Adventure', 'Indie', 'RPG', 'Early Access'],
 730: ['Action', 'Free To Play'],
 1599340: ['Action',
  'Adventure',
  'Massively Multiplayer',
  'RPG',
  'Free To Play'],
 570: ['Action', 'Strategy', 'Free To Play'],
 1091500: ['RPG'],
 1245620: ['Action', 'RPG'],
 2923300: ['Adventure', 'Casual', 'Simulation', 'Strategy', 'Free To Play'],
 1063730: ['Action', 'Adventure', 'Massively Multiplayer', 'RPG'],
 990080: ['Action', 'Adventure', 'RPG'],
 1086940: ['Adventure', 'RPG', 'Strategy'],
 1568590: ['Casual',
  'Indie',
  'Massively Multiplayer',
  'Strategy',
  'Free To Play'],
 1172470: ['Action', 'Adventure', 'Free To Play'],
 892970: ['Action', 'Adventure', 'Indie', 'RPG', 'Early Access'],
 1938090: ['Action'],
 105600: ['Action', 'Adventure', 'Indie', 'RPG'],
 1515950: ['Action', 'Free To Play'],
 377160: ['RPG'],
 532210: ['Adventure'],
 175591

In [119]:
appid_fix["tags"] = 0

#steam_df_tags = stf.add_tags(appid_fix,tag_dict)

In [120]:
with open ("steam_df_tags.pkl", "rb") as steam_df_tags_file:
    steam_df_tags = pickle.load(steam_df_tags_file)

steam_df_tags

,Game Title,Steam App ID,tags
0,PUBG: BATTLEGROUNDS,578080,"Action, Adventure, Massively Multiplayer, Free..."
1,Black Myth: Wukong,2358720,"Action, Adventure, RPG"
2,Palworld,1623730,"Action, Adventure, Indie, RPG, Early Access"
3,Counter-Strike 2,730,"Action, Free To Play"
4,Lost Ark,1599340,"Action, Adventure, Massively Multiplayer, RPG,..."
...,...,...,...
995,Winter Memories,2495450,Not Found
996,Magicraft,2103140,"Action, Adventure, Indie, RPG, Early Access"
997,Assetto Corsa Competizione,805550,"Racing, Simulation, Sports"
998,Timberborn,1062090,"Indie, Simulation, Strategy, Early Access"


In [121]:
#steam_df_tags.to_csv('steam_appid_tags.csv', index=False)

In [122]:
token = "1AC3575ADB761319F3EF72D1FA362232"

In [123]:
#mp_dict = stf.check_multiplayer(appid_list, token, delay=1)

In [124]:
with open ("mp_dict.pkl", "rb") as mp_dict_file:
    mp_dict = pickle.load(mp_dict_file)

mp_dict

{578080: ['Multi-player'],
 2358720: [],
 1623730: ['Multi-player', 'Co-op'],
 730: ['Multi-player'],
 1599340: ['Multi-player', 'Co-op'],
 570: ['Multi-player', 'Co-op'],
 1091500: [],
 1245620: ['Multi-player', 'Co-op'],
 2923300: [],
 1063730: ['Multi-player', 'Co-op'],
 990080: [],
 1086940: ['Multi-player', 'Co-op'],
 1568590: ['Multi-player', 'Co-op'],
 1172470: ['Multi-player', 'Co-op'],
 892970: ['Multi-player', 'Co-op'],
 1938090: ['Multi-player', 'Co-op'],
 105600: ['Multi-player', 'Co-op'],
 1515950: ['Multi-player', 'Co-op'],
 377160: [],
 532210: [],
 1755910: ['Multi-player', 'Co-op'],
 553850: ['Multi-player', 'Co-op'],
 945360: ['Multi-player', 'Co-op'],
 1326470: ['Multi-player', 'Co-op'],
 232770: [],
 552990: ['Multi-player', 'Co-op'],
 1203220: ['Multi-player'],
 271590: ['Multi-player', 'Co-op'],
 863550: ['Multi-player', 'Co-op'],
 370910: [],
 582010: ['Multi-player', 'Co-op'],
 1716740: [],
 622590: [],
 10: ['Multi-player'],
 1085660: ['Multi-player', 'Co-op'],

In [125]:
steam_df_tags_mp = stf.add_mp(steam_df_tags, mp_dict)

C:\Users\JimiH\IRON HACK\WEEK 3\Project-3\steam_tags_f.py:139: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  id = row[1]


In [152]:
steam_df_tags_mp.head()

,Game Title,Steam App ID,tags
0,PUBG: BATTLEGROUNDS,578080,"Action, Adventure, Massively Multiplayer, Free..."
1,Black Myth: Wukong,2358720,"Action, Adventure, RPG"
2,Palworld,1623730,"Action, Adventure, Indie, RPG, Early Access, M..."
3,Counter-Strike 2,730,"Action, Free To Play, Multi-player"
4,Lost Ark,1599340,"Action, Adventure, Massively Multiplayer, RPG,..."


In [ ]:
#steam_df_tags_mp.to_csv('steam_appid_tags_mp.csv', index=False)

In [ ]:
tags_mp_dict = steam_df_tags_mp.set_index("Steam App ID")["tags"].to_dict()

In [ ]:
tags_mp_dict

In [ ]:
with open ("tags_mp_dict.pkl", "wb") as tags_mp_dict_file:
    pickle.dump(tags_mp_dict, tags_mp_dict_file)

In [ ]:
tags_name_dict = steam_df_tags_mp.set_index("Steam App ID")["Game Title"].to_dict()

In [ ]:
tags_name_dict

In [ ]:
with open ("tags_name_dict.pkl", "wb") as tags_name_dict_file:
    pickle.dump(tags_name_dict, tags_name_dict_file)

FUNCTIONS

In [ ]:
def get_url(url, headers=None, params=None):
    
    response = requests.get(url,headers=headers, params=params)
    stat = response.status_code
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(1)
    
    return stat, soup

MAIN CODES

In [ ]:
gameid = "582660"
url = f'https://steamcharts.com/app/{gameid}'
headers= {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

stat, soup = get_url(url, headers=headers, params=gameid)
print(f"Status is : {stat}")

In [ ]:
k_value = 0    
game_id_dict = {}
player_game = {} 

players = soup.find("tbody")
play = re.split(r"[\n\t]+",players.get_text().strip())

for i in range(0, len(play), 5):
    # Date
    date = play[i]
    # Peak players
    peak = play[i+1]  
    # Average players
    average = play[i +4]
    player_game[k_value]={"Date": date,
                          "Peak Players":peak,
                          "Avg Players":average,
    }
    k_value += 1
    
game_id_dict[gameid] = player_game.copy()

game_id_dict
